# Urban Mobility

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/mobility.ipynb)

In [ ]:
import hvplot.pandas
from utils.dataaccess import get_uber_movement_gdf

## Uber Movement

Source: https://movement.uber.com/explore/vienna/travel-times

© 2021 Copyright Uber Technologies. Data made available under the [Creative Commons, Attribution Non-Commercial](https://creativecommons.org/licenses/by-nc/3.0/us/) license

In [ ]:
gdf = get_uber_movement_gdf()

In [ ]:
ID = 3
COL = 'mean_travel_time'
MONTH = 1
( 
    gdf[(gdf.sourceid==ID) & (gdf.month==MONTH)].dropna(subset=[COL]).hvplot(
        geo=True, tiles='OSM', c=COL, title=f'{COL.title()} - Month: 2020-{MONTH}') * 
    gdf[gdf.index==ID].hvplot(geo=True).opts(active_tools=['wheel_zoom'])
)